In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LassoLarsCV
import random
import pickle as pkl

In [22]:
random.seed(24)

In [23]:
eco_fac = pd.read_csv('eco_japan.csv')
eco_fac.head()

,,GDP_US,INFL_US,GDP,PPP,INFL,FER
0,1981,13976.1,10.334710,10361.3,228.831006,4.912163,220.535833
1,1982,14433.8,6.131427,9578.1,219.276639,2.741041,249.076667
2,1983,15543.9,3.212435,10425.4,213.007241,1.899720,237.511667
3,1984,17121.2,4.300536,10984.9,208.619257,2.261614,237.522500
4,1985,18236.8,3.545644,11584.6,204.772357,2.032277,238.535833


In [24]:
df = pd.read_csv('weights_15.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.025855,0.125928,0.054118,0.067509,0.037793,0.062103,0.057768,0.093079,0.032104,0.069524,0.053799,0.089232,0.020576,0.195674,0.014940
1,0.024390,0.107464,0.049101,0.059169,0.034891,0.060548,0.056063,0.094063,0.027917,0.074106,0.045950,0.088189,0.022177,0.239667,0.016304
2,0.025450,0.076065,0.035598,0.047638,0.028261,0.041214,0.046687,0.079440,0.030794,0.057839,0.034091,0.071603,0.016388,0.389751,0.019181
3,0.026685,0.087435,0.042064,0.061242,0.031366,0.051220,0.059974,0.104375,0.038531,0.068460,0.035101,0.074283,0.016077,0.283385,0.019803
4,0.028514,0.077057,0.046638,0.060663,0.031233,0.060692,0.060507,0.107761,0.041098,0.076824,0.038589,0.085528,0.015696,0.251406,0.017794


In [25]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

In [26]:
new_fer = eco_fac.FER.shift(-1)
new_fer = pd.DataFrame(new_fer)
new_fer.rename(columns = {'FER':'NEXT_YR_FER'}, inplace=True)

In [27]:
data = pd.concat([eco_fac,new_fer],axis=1)
data.head()

,,GDP_US,INFL_US,GDP,PPP,INFL,FER,NEXT_YR_FER
0,1981,13976.1,10.334710,10361.3,228.831006,4.912163,220.535833,249.076667
1,1982,14433.8,6.131427,9578.1,219.276639,2.741041,249.076667,237.511667
2,1983,15543.9,3.212435,10425.4,213.007241,1.899720,237.511667,237.522500
3,1984,17121.2,4.300536,10984.9,208.619257,2.261614,237.522500,238.535833
4,1985,18236.8,3.545644,11584.6,204.772357,2.032277,238.535833,168.519833


In [29]:
data = pd.concat([df,data],axis=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,GDP,FER,NEXT_YR_FER
0,0.025855,0.125928,0.054118,0.067509,0.037793,0.062103,0.057768,0.093079,0.032104,0.069524,0.053799,0.089232,0.020576,0.195674,0.014940,10361.3,220.535833,249.076667
1,0.024390,0.107464,0.049101,0.059169,0.034891,0.060548,0.056063,0.094063,0.027917,0.074106,0.045950,0.088189,0.022177,0.239667,0.016304,9578.1,249.076667,237.511667
2,0.025450,0.076065,0.035598,0.047638,0.028261,0.041214,0.046687,0.079440,0.030794,0.057839,0.034091,0.071603,0.016388,0.389751,0.019181,10425.4,237.511667,237.522500
3,0.026685,0.087435,0.042064,0.061242,0.031366,0.051220,0.059974,0.104375,0.038531,0.068460,0.035101,0.074283,0.016077,0.283385,0.019803,10984.9,237.522500,238.535833
4,0.028514,0.077057,0.046638,0.060663,0.031233,0.060692,0.060507,0.107761,0.041098,0.076824,0.038589,0.085528,0.015696,0.251406,0.017794,11584.6,238.535833,168.519833


In [30]:
data = data.dropna()

In [31]:
data.shape

(38, 18)

In [32]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [34]:
minmax = MinMaxScaler()
scaler = StandardScaler()

In [35]:
X_s = scaler.fit_transform(X) # standard scaled data
X_m = minmax.fit_transform(X) # minmax scaled data

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)
X_train_s, X_test_s, y_train, y_test = train_test_split(X_s, y, test_size=0.2, random_state=None)
X_train_m, X_test_m, y_train, y_test = train_test_split(X_m, y, test_size=0.2, random_state=None)

In [37]:
# Change model here

from sklearn.model_selection import cross_val_score
score = cross_val_score(LassoLarsCV(normalize=False), X_train, y_train, cv=5, scoring='neg_mean_squared_error')
score.mean()

-2475.4894446089743

In [38]:
# Change model here

score = cross_val_score(LassoLarsCV(normalize=False), X_train_m, y_train, cv=5, scoring='neg_mean_squared_error')
score.mean()

-395.0515939258618

In [39]:
# Change model here

score = cross_val_score(LassoLarsCV(normalize=False), X_train_s, y_train, cv=5, scoring='neg_mean_squared_error')
score.mean()

-2413.6899859294626

In [40]:
# Original 
model_orig = LassoLarsCV(normalize=False) # change model here
model_orig.fit(X_train, y_train)
pred = model_orig.predict(X_test)
print('MSE :',mean_squared_error(pred, y_test))
print('R2 :',r2_score(pred,y_test))
print('Error : ',((y_test-pred)/y_test).mean())

MSE : 323.36330124541576
R2 : 0.0
Error :  -0.12285576922952905


In [41]:
(y_test-pred) # check the variations

32   -24.786276
6     -2.579390
21   -14.797594
37   -21.721391
18   -22.965559
11   -19.533271
8     14.061442
25   -12.977528
Name: NEXT_YR_FER, dtype: float64

In [42]:
# MinMax Scaled 
model_minmax = LassoLarsCV(normalize=False) # Change model here
model_minmax.fit(X_train_m, y_train)
pred = model_minmax.predict(X_test_m)
print('MSE :',mean_squared_error(pred, y_test))
print('R2 :',r2_score(pred,y_test))
print('Error : ',((y_test-pred)/y_test).mean())

MSE : 79.72392753658737
R2 : 0.17066712388718108
Error :  -0.036398952611313234


In [43]:
(y_test-pred) # check the variation

32     2.057740
6     -8.974061
21    -9.320060
37    -8.501103
18    -7.391417
11   -13.022689
8     12.916632
25     1.681726
Name: NEXT_YR_FER, dtype: float64

In [44]:
# Standard Scaled 
model_stand = LassoLarsCV(normalize=False) # Change model here
model_stand.fit(X_train_s, y_train)
pred = model_stand.predict(X_test_s)
print('MSE :',mean_squared_error(pred, y_test))
print('R2 :',r2_score(pred,y_test))
print('Error : ',((y_test-pred)/y_test).mean())

MSE : 323.36330124541576
R2 : 0.0
Error :  -0.12285576922952905


In [45]:
(y_test-pred) # check the variations

32   -24.786276
6     -2.579390
21   -14.797594
37   -21.721391
18   -22.965559
11   -19.533271
8     14.061442
25   -12.977528
Name: NEXT_YR_FER, dtype: float64

In [48]:
# Save the best perform ming model only after instructions

pkl.dump(model_minmax, open('model_japan.pkl','wb'))

In [49]:
# dont uncomment until said to
# if we use scaling
pkl.dump(minmax, open('scaler_japan.pkl','wb'))